In [1]:
from game import Game2048
from game_env import Game2048Env

# Search content

In [ ]:
# Definir modelo de NN por reforço
    # Métodos
        # Q-Learning
        # Monte Carlo
        # Gradiente de políticas
        # Aprendizado por diferença temporal
        # Trust Region Policy Optimization (TRPO)
# Definir parâmetros
    # Grid do jogo
# Definir penalidades
    # Tentar mover para uma direção sem movimentos válidos
    # Valores iguais/próximos muito dispersos? (viés?)
        # Aplicar penalidade apenas quando saiu do range (até 2 de distância?)
        # Verificar valores disponíveis no grid e contabilizar como valor próximo se estiver a até (3?) valores de distância
        # Não penalizar se valor for 2 ou 4.
# Definir recompensas
    # Juntar tiles
    # Manter valores da sequência em fileira ou próximos? (viés?)
    # 
# Definir estratégia de treino
    # Utilizar jogo já iniciado com mais tiles no grid? (aumentar aprendizado inicial sobre a regra do jogo)
        # Treinar o mesmo jogo com rotações diferentes

# Material:
# https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc
# https://www.analyticsvidhya.com/blog/2019/04/introduction-deep-q-learning-python
# https://www.analyticsvidhya.com/blog/2019/01/monte-carlo-tree-search-introduction-algorithm-deepmind-alphago
# https://medium.com/@samina.amin/deep-q-learning-dqn-71c109586bae
# https://www.geeksforgeeks.org/implementing-deep-q-learning-using-tensorflow/

# Model

In [2]:
# pip install tensorflow==2.9.0 keras-rl2
# pip uninstall numpy -y
# pip install numpy==1.24.3

import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam, RMSprop

from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

2025-03-28 00:09:03.732772: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-28 00:09:03.732815: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:

nActions = 4
tableSize = 4
# hiddenFeatures = 16
hiddenFeatures = 32

activation="relu"
# activation="leaky_relu"

# Higher temporal window confuses the model
# temporalWindow = 5
temporalWindow = 1



env = Game2048Env()

model = Sequential()
model.add(Flatten(input_shape=(temporalWindow, tableSize, tableSize)))

model.add(Dense(hiddenFeatures))

# Result look alike
model.add(Activation(activation=activation))
# model.add(Activation(activation="relu"))
# model.add(Activation(activation="leaky_relu")) # Tune alpha

# Worst result
# model.add(Activation(activation="elu")) # Tune alpha


# Additional layers
# model.add(Dense(hiddenFeatures))
# model.add(Activation(activation=activation))

# model.add(Dense(hiddenFeatures))
# model.add(Activation(activation=activation))


# Output layer
model.add(Dense(nActions))
model.add(Activation(activation="linear"))

policy = EpsGreedyQPolicy()

# Less invalid moves with more limit
replayMemory = SequentialMemory(limit=2000, window_length=temporalWindow)

dqn = DQNAgent(
    model=model,
    nb_actions=nActions,
    memory=replayMemory,
    nb_steps_warmup=1000,
    target_model_update=0.01,
    policy=policy,
)
# dqn.compile(optimizer=Adam(learning_rate=0.001), metrics=["mae"])
# dqn.compile(optimizer=Adam(learning_rate=0.001), metrics=["mse"])
dqn.compile(optimizer=RMSprop(learning_rate=0.001, epsilon=0.0000001), metrics=["mae"])
# dqn.compile(optimizer=RMSprop(learning_rate=0.001, epsilon=0.0000001), metrics=["mse"])
dqn.fit(env=env, nb_steps=20000, visualize=False, verbose=2)


Training for 20000 steps ...


/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   406/20000: episode: 1, duration: 0.388s, episode steps: 406, steps per second: 1048, episode reward: -14842.000, mean reward: -36.557 [-50.000, 84.000], mean action: 2.069 [0.000, 3.000],  loss: --, mae: --, mean_q: --
   915/20000: episode: 2, duration: 0.329s, episode steps: 509, steps per second: 1547, episode reward: -18660.000, mean reward: -36.660 [-50.000, 128.000], mean action: 2.102 [0.000, 3.000],  loss: --, mae: --, mean_q: --


/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  1488/20000: episode: 3, duration: 2.814s, episode steps: 573, steps per second: 204, episode reward: -17998.000, mean reward: -31.410 [-50.000, 260.000], mean action: 0.990 [0.000, 3.000],  loss: 564.431475, mae: 26.227176, mean_q: -20.476298
  1679/20000: episode: 4, duration: 0.879s, episode steps: 191, steps per second: 217, episode reward: -5678.000, mean reward: -29.728 [-50.000, 80.000], mean action: 0.649 [0.000, 3.000],  loss: 374.835358, mae: 63.950764, mean_q: -73.039307
  2049/20000: episode: 5, duration: 1.609s, episode steps: 370, steps per second: 230, episode reward: -11810.000, mean reward: -31.919 [-50.000, 140.000], mean action: 2.297 [0.000, 3.000],  loss: 408.101562, mae: 99.414070, mean_q: -120.218475
  2286/20000: episode: 6, duration: 1.038s, episode steps: 237, steps per second: 228, episode reward: -4356.000, mean reward: -18.380 [-50.000, 136.000], mean action: 1.713 [0.000, 3.000],  loss: 486.465546, mae: 144.610794, mean_q: -179.926620
  2387/20000: episod

# Simulate Game

In [26]:
# Play the game with predicts

def log2_transform(grid):
    transformed = np.log2(np.where(grid > 0, grid, 1))
    return transformed

actionMapping = {
    0: "up",
    1: "down",
    2: "left",
    3: "right"
}

games = [Game2048() for _ in range(100)]
metrics = {
    "gameId": [],
    "score": [],
    "higherTile": [],
    "invalidMove (%)": [],
    "steps": [],
    "upMoves": [],
    "downMoves": [],
    "leftMoves": [],
    "rightMoves": [],
    "upSeqAvg": [],
    "downSeqAvg": [],
    "leftSeqAvg": [],
    "rightSeqAvg": [],
}
for i, game in enumerate(games):
    # Metrics
    steps = 0
    invalidMove = 0
    upMoves = 0
    downMoves = 0
    leftMoves = 0
    rightMoves = 0
    upSeqAvg = 0
    downSeqAvg = 0
    leftSeqAvg = 0
    rightSeqAvg = 0

    sequences = {
        "up": [],
        "down": [],
        "left": [],
        "right": []
    }
    lastMove = None

    # print(f"Game {i + 1} initial state:")
    # game.render()
    while (not game.game_over):
        observation = np.expand_dims(game.grid, axis=0)[0]
        observation = log2_transform(observation)
        predictedAction = dqn.forward(observation)
        
        # print(f"\nPredicted Action: move {actionMapping[predictedAction]}")
        if not game.move(actionMapping[predictedAction]):
            invalidMove += 1
        
        # print(f"Game {i + 1} currrent state:")
        # game.render()

        # input("\nPress any key to continue...")

        steps += 1
        if actionMapping[predictedAction] == "up":
            upMoves += 1
            if lastMove == "up":
                sequences["up"][-1] += 1
            else:
                sequences["up"].append(1)
        elif actionMapping[predictedAction] == "down":
            downMoves += 1
            if lastMove == "down":
                sequences["down"][-1] += 1
            else:
                sequences["down"].append(1)
        elif actionMapping[predictedAction] == "left":
            leftMoves += 1
            if lastMove == "left":
                sequences["left"][-1] += 1
            else:
                sequences["left"].append(1)
        elif actionMapping[predictedAction] == "right":
            rightMoves += 1
            if lastMove == "right":
                sequences["right"][-1] += 1
            else:
                sequences["right"].append(1)
        
        lastMove = actionMapping[predictedAction]

    if (i+1) % 10 == 0:
        print(f"Game {i + 1} is over.")

    metrics["gameId"].append(i)
    metrics["score"].append(game.score)
    metrics["higherTile"].append(np.max(game.grid))
    metrics["invalidMove (%)"].append(invalidMove/steps)
    metrics["steps"].append(steps)
    metrics["upMoves"].append(upMoves)
    metrics["downMoves"].append(downMoves)
    metrics["leftMoves"].append(leftMoves)
    metrics["rightMoves"].append(rightMoves)
    metrics["upSeqAvg"].append(np.mean(np.array(sequences["up"])))
    metrics["downSeqAvg"].append(np.mean(np.array(sequences["down"])))
    metrics["leftSeqAvg"].append(np.mean(np.array(sequences["left"])))
    metrics["rightSeqAvg"].append(np.mean(np.array(sequences["right"])))


metrics = pd.DataFrame(metrics).set_index("gameId")
# display(metrics)
display(metrics.describe())


Game 10 is over.
Game 20 is over.
Game 30 is over.
Game 40 is over.
Game 50 is over.
Game 60 is over.
Game 70 is over.
Game 80 is over.
Game 90 is over.
Game 100 is over.


,score,higherTile,invalidMove (%),steps,upMoves,downMoves,leftMoves,rightMoves,upSeqAvg,downSeqAvg,leftSeqAvg,rightSeqAvg
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,1130.440000,117.120000,0.262594,176.280000,41.710000,39.330000,54.070000,41.170000,1.508777,1.339330,1.660872,1.329625
std,579.360817,64.678424,0.185439,85.838359,35.783946,28.395851,44.392022,28.464422,1.100372,0.620959,0.965732,0.738339
min,268.000000,32.000000,0.000000,53.000000,12.000000,9.000000,13.000000,12.000000,1.000000,1.000000,1.000000,1.000000
25%,635.000000,64.000000,0.094694,113.000000,22.000000,22.750000,28.000000,23.750000,1.035406,1.029848,1.027590,1.000000
50%,1086.000000,128.000000,0.241128,156.000000,29.500000,30.000000,40.000000,34.000000,1.101959,1.074176,1.178409,1.061862
75%,1393.000000,128.000000,0.418411,212.000000,43.000000,43.000000,70.500000,50.000000,1.285256,1.317726,2.000000,1.211806
max,2484.000000,256.000000,0.683721,445.000000,224.000000,162.000000,309.000000,177.000000,6.789474,4.378378,5.150000,5.740741


# Manual Game

In [ ]:
moves = ["up", "left", "right", "down"]
directions = ["w", "a", "d", "s"]

games = [Game2048() for _ in range(1)]
for i, game in enumerate(games):
    print(f"Game {i + 1} initial state:")
    game.render()
    while (not game.game_over):
        try: 
            currentMove = moves[directions.index(input("Next move (w, a, s , d): "))]
        except ValueError:
            print("Invalid move, try again.")
            continue

        game.move(currentMove)
        print(f"Game {i + 1} after move '{currentMove}':")
        game.render()

    print(f"Game {i + 1} is over.")


